# Equipment_move_workflow

In [ ]:
## FROM : Ancienne table, INNER JOIN : Nouvelle base 

/*delimiter //
CREATE TRIGGER `move_equipment` BEFORE INSERT ON `IT_Equipment_records`
 FOR EACH ROW BEGIN
    DECLARE _Reperage_ID , _Nom_de_la_salle, _INSERTED varchar(255);
    
        SELECT Reperage_ID, Nom_de_la_salle, INSERTED 
        INTO _Reperage_ID, _Nom_de_la_salle, _INSERTED
        FROM `IT_Equipment_records`
        WHERE ID_Systeme_Asset = NEW.ID_Systeme_Asset
        ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') DESC
        LIMIT 1 ;
    
        IF(
          (NEW.reperage_id != _Reperage_ID OR 
          NEW.Nom_de_la_salle != _Nom_de_la_salle) AND 
          ( NEW.ID_Systeme_Asset != '0' or NEW.ID_Systeme_Asset != ' ' )
        )
        THEN
        INSERT INTO move_records_equipments (asset_id, ID_Systeme_Asset, modele, from_salle, from_reperage_id, to_salle, to_reperage_id, from_date, to_date) 
           VALUES (NEW.asset_id, NEW.ID_Systeme_Asset, NEW.modele, _Nom_de_la_salle, _reperage_id, new.Nom_de_la_salle, new.reperage_id, _INSERTED, new.inserted);
        END IF;
           
END
*/

In [ ]:
drop view if exists IT_Equipment_sorted_by_date_asc ;
create view `IT_Equipment_sorted_by_date_asc` 
as
select ASSET_ID, ID_Systeme_Asset, Modele, Nom_de_la_salle, Reperage_ID, SOURCES, INSERTED from `IT_Equipment_records` 
ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') ASC ;

drop view if exists IT_Equipment_sorted_by_date_desc ;
create view `IT_Equipment_sorted_by_date_desc` 
as
select ASSET_ID, ID_Systeme_Asset, Modele, Nom_de_la_salle, Reperage_ID, SOURCES, INSERTED from `IT_Equipment_records` 
ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') DESC ;

1238131-88444

In [ ]:
drop view if exists IT_Equipment_first_last_occurence_per_id_all ;
CREATE view IT_Equipment_first_last_occurence_per_id_all
as 
select min(inserted) first_records, max(inserted) last_records, it.* from `IT_Equipment_sorted_by_date_asc` it
GROUP BY ID_Systeme_Asset
order by INSERTED desc ;

In [ ]:
drop view if exists IT_Equipment_first_occurence_per_id ;
CREATE view  IT_Equipment_first_occurence_per_id
as
select it.* from `IT_Equipment_records` it
INNER JOIN `IT_Equipment_first_last_occurence_per_id_all` it2 
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset
AND it.inserted = it2.first_records ;

drop view if exists IT_Equipment_last_occurence_per_id ;
CREATE view  IT_Equipment_last_occurence_per_id
as
select it.* from `IT_Equipment_records` it
INNER JOIN `IT_Equipment_first_last_occurence_per_id_all` it2 
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset
AND it.inserted = it2.last_records ;

In [ ]:
select it.nom, it.Nom_de_la_salle, it.Reperage_ID, it.INSERTED, it.first_occurence, it.last_move, it.last_move_date,
(CASE WHEN concat(it2.nom_de_la_salle,it2.reperage_id)  != concat(it3.nom_de_la_salle,it3.reperage_id) and it3.reperage_id != ''  THEN it3.reperage_id
WHEN concat(it2.nom_de_la_salle,it2.reperage_id)  = concat(it3.nom_de_la_salle,it3.reperage_id) and it3.reperage_id != ''  THEN it2.reperage_id
WHEN it3.reperage_id = ''  then it2.reperage_id
END) NEW_POS

from `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_first_occurence_per_id` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

INNER JOIN `IT_Equipment_last_occurence_per_id` it3
ON it.ID_Systeme_Asset = it3.ID_Systeme_Asset

where it.ID_Systeme_Asset = '1238131-88444'

In [ ]:
drop view if exists IT_Equipment_last_occurence_position ;
CREATE view  IT_Equipment_last_occurence_position
as
SELECT it2.ID_Systeme_Asset, it2.Reperage_ID, it2.INSERTED
FROM `IT_Equipment_last_occurence_per_id` it
INNER JOIN `IT_Equipment_records` it2 ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset AND it.reperage_id = it2.reperage_id
GROUP BY it2.ID_Systeme_Asset

In [ ]:
drop view if EXISTS last_date_insertion ;
create view last_date_insertion
as
select DISTINCT(INSERTED) source_insert from `IT_Equipment_records`
ORDER by INSERTED desc
limit 1

# First occurence

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_first_occurence_per_id` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set first_occurence = it2.inserted

# Last move

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_last_occurence_position` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set last_move_date = it2.INSERTED ,
    last_move = it2.Reperage_ID